In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score


from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [2]:
df = pd.read_csv('../data/full_data_interaction_05_025.csv')

In [3]:
X = df.drop(columns = ['state', 'county_name', 'unemployment_rate_2010', 'population_total_2010'])
y = df['unemployment_rate_2010']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

### Linear Regression

In [6]:
lr = LinearRegression()
lr.fit(Z_train, y_train)

LinearRegression()

In [7]:
lr.score(Z_train, y_train), lr.score(Z_test, y_test), cross_val_score(lr, Z_train, y_train).mean()

(0.5440914846052834, 0.45134894131845105, 0.4834601772545518)

In [8]:
lin_train = lr.predict(Z_train)
lin_test = lr.predict(Z_test)

### PCA data Linear Regression

In [9]:
df = pd.read_csv('../data/final_work_data.csv')

In [10]:
X = df.drop(columns = ['state', 'county_name', 'unemployment_rate_2010', 'population_total_2010'])
y = df['unemployment_rate_2010']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [12]:
pipe = Pipeline([
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('ss', StandardScaler()),
    ('pca', PCA()),
    ('model', LinearRegression())
])
params = {
    'pca__n_components': range(10,200,20)
}
gs = GridSearchCV(pipe, param_grid=params)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.score(X_test, y_test))
print(cross_val_score(gs, X_train, y_train).mean())
gs.best_params_

0.4009664055310037
0.38311006597792363
0.38971682481060954


{'pca__n_components': 30}

### Neural network on Linear Regression initeraction data

In [13]:
df = pd.read_csv('../data/final_work_data.csv')

In [14]:
X = df.drop(columns = ['state', 'county_name', 'unemployment_rate_2010', 'population_total_2010'])
y = df['unemployment_rate_2010']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [16]:
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

In [17]:
in_shape = Z_train[0].shape
in_shape

(24,)

In [18]:
model_1 = Sequential()

model_1.add(Dense(128, activation= 'relu', input_shape= in_shape))
model_1.add(Dropout(.3))

model_1.add(BatchNormalization())
model_1.add(Dense(128, activation= 'relu'))
"""
model_1.add(BatchNormalization())
model_1.add(Dense(64, activation= 'relu'))
"""
model_1.add(Dense(32, activation= 'relu'))
model_1.add(Dropout(.5))

model_1.add(Dense(1, activation= None))

In [19]:
model_1.compile(optimizer= 'adam', loss= 'mse', metrics= ['mse', 'RootMeanSquaredError', 'MeanAbsolutePercentageError'])

In [20]:
history_1 = model_1.fit(Z_train, y_train,
          epochs= 300,
          validation_data= (Z_test, y_test),
          verbose= 1
         )

Epoch 1/300
62/62 [==============================] - 1s 8ms/step - loss: 32.5897 - mse: 32.5897 - root_mean_squared_error: 5.7087 - mean_absolute_percentage_error: 48.9824 - val_loss: 46.5191 - val_mse: 46.5191 - val_root_mean_squared_error: 6.8205 - val_mean_absolute_percentage_error: 66.0996
Epoch 2/300
62/62 [==============================] - 0s 5ms/step - loss: 12.7986 - mse: 12.7986 - root_mean_squared_error: 3.5775 - mean_absolute_percentage_error: 31.6981 - val_loss: 34.9913 - val_mse: 34.9913 - val_root_mean_squared_error: 5.9153 - val_mean_absolute_percentage_error: 56.3793
Epoch 3/300
62/62 [==============================] - 0s 4ms/step - loss: 10.9293 - mse: 10.9293 - root_mean_squared_error: 3.3060 - mean_absolute_percentage_error: 29.4155 - val_loss: 23.2726 - val_mse: 23.2726 - val_root_mean_squared_error: 4.8242 - val_mean_absolute_percentage_error: 44.9460
Epoch 4/300
62/62 [==============================] - 0s 4ms/step - loss: 11.2765 - mse: 11.2765 - root_mean_squared

62/62 [==============================] - 0s 3ms/step - loss: 7.4038 - mse: 7.4038 - root_mean_squared_error: 2.7210 - mean_absolute_percentage_error: 23.5170 - val_loss: 6.2256 - val_mse: 6.2256 - val_root_mean_squared_error: 2.4951 - val_mean_absolute_percentage_error: 21.0252
Epoch 30/300
62/62 [==============================] - 0s 4ms/step - loss: 7.4849 - mse: 7.4849 - root_mean_squared_error: 2.7358 - mean_absolute_percentage_error: 23.8690 - val_loss: 5.5884 - val_mse: 5.5884 - val_root_mean_squared_error: 2.3640 - val_mean_absolute_percentage_error: 19.3219
Epoch 31/300
62/62 [==============================] - 0s 5ms/step - loss: 7.3305 - mse: 7.3305 - root_mean_squared_error: 2.7075 - mean_absolute_percentage_error: 23.1447 - val_loss: 5.0660 - val_mse: 5.0660 - val_root_mean_squared_error: 2.2508 - val_mean_absolute_percentage_error: 18.8544
Epoch 32/300
62/62 [==============================] - 0s 5ms/step - loss: 7.1317 - mse: 7.1317 - root_mean_squared_error: 2.6705 - mean_a

62/62 [==============================] - 0s 3ms/step - loss: 6.7741 - mse: 6.7741 - root_mean_squared_error: 2.6027 - mean_absolute_percentage_error: 22.2476 - val_loss: 4.2363 - val_mse: 4.2363 - val_root_mean_squared_error: 2.0582 - val_mean_absolute_percentage_error: 17.2482
Epoch 58/300
62/62 [==============================] - 0s 3ms/step - loss: 6.4999 - mse: 6.4999 - root_mean_squared_error: 2.5495 - mean_absolute_percentage_error: 21.8744 - val_loss: 4.2106 - val_mse: 4.2106 - val_root_mean_squared_error: 2.0520 - val_mean_absolute_percentage_error: 17.3624
Epoch 59/300
62/62 [==============================] - 0s 4ms/step - loss: 6.3079 - mse: 6.3079 - root_mean_squared_error: 2.5116 - mean_absolute_percentage_error: 21.1820 - val_loss: 5.5822 - val_mse: 5.5822 - val_root_mean_squared_error: 2.3627 - val_mean_absolute_percentage_error: 19.9995
Epoch 60/300
62/62 [==============================] - 0s 4ms/step - loss: 6.2492 - mse: 6.2492 - root_mean_squared_error: 2.4998 - mean_a

62/62 [==============================] - 0s 5ms/step - loss: 5.3992 - mse: 5.3992 - root_mean_squared_error: 2.3236 - mean_absolute_percentage_error: 20.0632 - val_loss: 4.2728 - val_mse: 4.2728 - val_root_mean_squared_error: 2.0671 - val_mean_absolute_percentage_error: 17.3761
Epoch 86/300
62/62 [==============================] - 0s 3ms/step - loss: 5.2169 - mse: 5.2169 - root_mean_squared_error: 2.2841 - mean_absolute_percentage_error: 19.7833 - val_loss: 5.0736 - val_mse: 5.0736 - val_root_mean_squared_error: 2.2525 - val_mean_absolute_percentage_error: 18.9836
Epoch 87/300
62/62 [==============================] - 0s 3ms/step - loss: 5.8316 - mse: 5.8316 - root_mean_squared_error: 2.4149 - mean_absolute_percentage_error: 20.6574 - val_loss: 4.3201 - val_mse: 4.3201 - val_root_mean_squared_error: 2.0785 - val_mean_absolute_percentage_error: 17.8932
Epoch 88/300
62/62 [==============================] - 0s 5ms/step - loss: 5.7497 - mse: 5.7497 - root_mean_squared_error: 2.3978 - mean_a

62/62 [==============================] - 0s 4ms/step - loss: 5.0936 - mse: 5.0936 - root_mean_squared_error: 2.2569 - mean_absolute_percentage_error: 18.9495 - val_loss: 4.2708 - val_mse: 4.2708 - val_root_mean_squared_error: 2.0666 - val_mean_absolute_percentage_error: 17.2990
Epoch 114/300
62/62 [==============================] - 0s 4ms/step - loss: 4.9303 - mse: 4.9303 - root_mean_squared_error: 2.2204 - mean_absolute_percentage_error: 18.9308 - val_loss: 4.4731 - val_mse: 4.4731 - val_root_mean_squared_error: 2.1150 - val_mean_absolute_percentage_error: 17.6308
Epoch 115/300
62/62 [==============================] - 0s 3ms/step - loss: 4.9483 - mse: 4.9483 - root_mean_squared_error: 2.2245 - mean_absolute_percentage_error: 18.7006 - val_loss: 4.6317 - val_mse: 4.6317 - val_root_mean_squared_error: 2.1521 - val_mean_absolute_percentage_error: 17.9226
Epoch 116/300
62/62 [==============================] - 0s 4ms/step - loss: 5.0745 - mse: 5.0745 - root_mean_squared_error: 2.2527 - mea

...

62/62 [==============================] - 0s 3ms/step - loss: 3.8257 - mse: 3.8257 - root_mean_squared_error: 1.9559 - mean_absolute_percentage_error: 16.6511 - val_loss: 3.8797 - val_mse: 3.8797 - val_root_mean_squared_error: 1.9697 - val_mean_absolute_percentage_error: 16.9238
Epoch 170/300
62/62 [==============================] - 0s 3ms/step - loss: 3.7829 - mse: 3.7829 - root_mean_squared_error: 1.9450 - mean_absolute_percentage_error: 16.5911 - val_loss: 4.3115 - val_mse: 4.3115 - val_root_mean_squared_error: 2.0764 - val_mean_absolute_percentage_error: 17.4287
Epoch 171/300
62/62 [==============================] - 0s 4ms/step - loss: 3.5854 - mse: 3.5854 - root_mean_squared_error: 1.8935 - mean_absolute_percentage_error: 16.0700 - val_loss: 4.0986 - val_mse: 4.0986 - val_root_mean_squared_error: 2.0245 - val_mean_absolute_percentage_error: 17.2409
Epoch 172/300
62/62 [==============================] - 0s 4ms/step - loss: 3.7359 - mse: 3.7359 - root_mean_squared_error: 1.9328 - mea

62/62 [==============================] - 0s 3ms/step - loss: 3.1273 - mse: 3.1273 - root_mean_squared_error: 1.7684 - mean_absolute_percentage_error: 15.6624 - val_loss: 4.5692 - val_mse: 4.5692 - val_root_mean_squared_error: 2.1376 - val_mean_absolute_percentage_error: 18.2138
Epoch 198/300
62/62 [==============================] - 0s 3ms/step - loss: 3.1617 - mse: 3.1617 - root_mean_squared_error: 1.7781 - mean_absolute_percentage_error: 15.2790 - val_loss: 3.8588 - val_mse: 3.8588 - val_root_mean_squared_error: 1.9644 - val_mean_absolute_percentage_error: 16.8221
Epoch 199/300
62/62 [==============================] - 0s 2ms/step - loss: 3.4348 - mse: 3.4348 - root_mean_squared_error: 1.8533 - mean_absolute_percentage_error: 16.2683 - val_loss: 4.3913 - val_mse: 4.3913 - val_root_mean_squared_error: 2.0955 - val_mean_absolute_percentage_error: 17.9247
Epoch 200/300
62/62 [==============================] - 0s 3ms/step - loss: 3.5514 - mse: 3.5514 - root_mean_squared_error: 1.8845 - mea

62/62 [==============================] - 0s 3ms/step - loss: 3.0725 - mse: 3.0725 - root_mean_squared_error: 1.7528 - mean_absolute_percentage_error: 15.2959 - val_loss: 4.0587 - val_mse: 4.0587 - val_root_mean_squared_error: 2.0146 - val_mean_absolute_percentage_error: 17.2360
Epoch 226/300
62/62 [==============================] - 0s 3ms/step - loss: 2.9476 - mse: 2.9476 - root_mean_squared_error: 1.7169 - mean_absolute_percentage_error: 15.1787 - val_loss: 3.8853 - val_mse: 3.8853 - val_root_mean_squared_error: 1.9711 - val_mean_absolute_percentage_error: 17.3177
Epoch 227/300
62/62 [==============================] - 0s 3ms/step - loss: 3.3201 - mse: 3.3201 - root_mean_squared_error: 1.8221 - mean_absolute_percentage_error: 15.5927 - val_loss: 4.1539 - val_mse: 4.1539 - val_root_mean_squared_error: 2.0381 - val_mean_absolute_percentage_error: 17.4809
Epoch 228/300
62/62 [==============================] - 0s 3ms/step - loss: 2.9710 - mse: 2.9710 - root_mean_squared_error: 1.7237 - mea

62/62 [==============================] - 0s 3ms/step - loss: 2.8332 - mse: 2.8332 - root_mean_squared_error: 1.6832 - mean_absolute_percentage_error: 14.7657 - val_loss: 3.9306 - val_mse: 3.9306 - val_root_mean_squared_error: 1.9826 - val_mean_absolute_percentage_error: 17.3565
Epoch 254/300
62/62 [==============================] - 0s 2ms/step - loss: 2.8622 - mse: 2.8622 - root_mean_squared_error: 1.6918 - mean_absolute_percentage_error: 15.2596 - val_loss: 3.9099 - val_mse: 3.9099 - val_root_mean_squared_error: 1.9774 - val_mean_absolute_percentage_error: 17.4272
Epoch 255/300
62/62 [==============================] - 0s 2ms/step - loss: 2.7789 - mse: 2.7789 - root_mean_squared_error: 1.6670 - mean_absolute_percentage_error: 15.2519 - val_loss: 4.0212 - val_mse: 4.0212 - val_root_mean_squared_error: 2.0053 - val_mean_absolute_percentage_error: 17.5185
Epoch 256/300
62/62 [==============================] - 0s 3ms/step - loss: 2.9417 - mse: 2.9417 - root_mean_squared_error: 1.7151 - mea

62/62 [==============================] - 0s 3ms/step - loss: 2.6157 - mse: 2.6157 - root_mean_squared_error: 1.6173 - mean_absolute_percentage_error: 14.2778 - val_loss: 3.9974 - val_mse: 3.9974 - val_root_mean_squared_error: 1.9994 - val_mean_absolute_percentage_error: 17.0629
Epoch 282/300
62/62 [==============================] - 0s 3ms/step - loss: 2.6620 - mse: 2.6620 - root_mean_squared_error: 1.6316 - mean_absolute_percentage_error: 14.6449 - val_loss: 4.1688 - val_mse: 4.1688 - val_root_mean_squared_error: 2.0418 - val_mean_absolute_percentage_error: 17.1885
Epoch 283/300
62/62 [==============================] - 0s 3ms/step - loss: 2.6828 - mse: 2.6828 - root_mean_squared_error: 1.6379 - mean_absolute_percentage_error: 14.3695 - val_loss: 4.1865 - val_mse: 4.1865 - val_root_mean_squared_error: 2.0461 - val_mean_absolute_percentage_error: 17.3493
Epoch 284/300
62/62 [==============================] - 0s 3ms/step - loss: 2.7780 - mse: 2.7780 - root_mean_squared_error: 1.6667 - mea

In [21]:
r2_score(y_train, model_1.predict(Z_train))

0.8495193980691386

In [22]:
r2_score(y_test, model_1.predict(Z_test))

0.5912298482775749

In [23]:
nn_train = model_1.predict(Z_train)
nn_test = model_1.predict(Z_test)
nn_train = [g[0] for g in nn_train]
nn_test = [h[0] for h in nn_test]

### Second level model based on linear regression and neural network preds

In [24]:
a =pd.DataFrame([lin_train, nn_train, y_train], index=['lin_reg', 'nn', 'y']).T
b = pd.DataFrame([lin_test, nn_test, y_test], index=['lin_reg', 'nn', 'y']).T

In [25]:
a.head()

,lin_reg,nn,y
0,14.491725,17.818363,17.3
1,9.733514,17.107672,18.7
2,10.720415,10.453696,11.3
3,9.052294,7.383136,8.1
4,9.663620,10.758892,12.6


In [26]:
Xa_train = a.drop(columns='y')
ya_train = a['y']

In [27]:
b.head()

,lin_reg,nn,y
0,8.579643,8.821253,8.7
1,13.718193,16.608334,14.2
2,9.979250,9.419676,10.7
3,7.657098,8.733862,10.0
4,7.761239,10.169096,9.5


In [28]:
Xb_test = b.drop(columns='y')
yb_test = b['y']

In [29]:
lev2 = LinearRegression()

In [30]:
lev2.fit(Xa_train, ya_train)

LinearRegression()

In [31]:
lev2.score(Xa_train, ya_train), lev2.score(Xb_test, yb_test), cross_val_score(lev2, Xa_train, ya_train).mean()

(0.9009880466719563, 0.6166400989225644, 0.8996610030098859)